In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *

In [15]:
# raju = ["idfcfirstb", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
sorted_stock = SortedStocksList.objects.get(symbol__symbol="l&tfh", created_at__date=datetime.now().date())

In [52]:
macd_indicator = Indicator.objects.get(name="MACD")
df = get_macd_data(sorted_stock.symbol)
today_date = datetime.today().date()
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df[76:]
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
except:
    crossover_signal = None
if crossover_signal is not None and last_crossover.date == today_date:
    stamp = {"stock": sorted_stock, "indicator": macd_indicator, "timestamp":crossover_signal.date}
# df.loc[(df.macd_diff <= -0.070)]
# df[df.signal.str.endswith("SELL_CROSSOVER")]
df

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,macd,macd_signal,macd_diff,percentage,signal
76,M5,119.75,119.75,119.10,119.55,118093,None,None,2020-01-30 09:15:00,-0.021622,-0.017048,-0.004574,0.000001,SELL_CROSSOVER
77,M5,119.50,119.60,118.80,118.85,116219,None,None,2020-01-30 09:20:00,-0.094787,-0.032596,-0.062191,0.000059,SELL
78,M5,118.90,119.00,118.70,118.70,92740,None,None,2020-01-30 09:25:00,-0.162969,-0.058671,-0.104298,0.000170,SELL
79,M5,118.80,118.80,118.15,118.45,248613,None,None,2020-01-30 09:30:00,-0.234432,-0.093823,-0.140608,0.000330,SELL
80,M5,118.45,118.55,117.95,118.45,146190,None,None,2020-01-30 09:35:00,-0.287747,-0.132608,-0.155139,0.000446,SELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,M5,117.10,117.20,117.00,117.05,120065,None,None,2020-01-30 15:05:00,-0.178436,-0.326841,0.148405,-0.000265,BUY
147,M5,117.05,117.25,116.90,117.15,308638,None,None,2020-01-30 15:10:00,-0.120290,-0.285531,0.165241,-0.000199,BUY
148,M5,117.10,117.15,116.85,117.05,296320,None,None,2020-01-30 15:15:00,-0.081341,-0.244693,0.163352,-0.000133,BUY
149,M5,117.00,117.15,116.70,116.70,392295,None,None,2020-01-30 15:20:00,-0.077817,-0.211318,0.133500,-0.000104,BUY


In [4]:
# macd_indicator = Indicator.objects.get(name="MACD")
# df = get_stochastic_data(sorted_stock.symbol)
# df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
# df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
# df = df[75:]
# if sorted_stock.entry_type in ["SELL", "SS"]:
#     last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
# elif sorted_stock.entry_type in ["BUY", "SB"]:
#     last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
# df_after_last_crossover = df.loc[df["date"] >= last_crossover.date]
# try:
#     if last_crossover.signal == "SELL_CROSSOVER":
#         crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
#     elif last_crossover.signal == "BUY_CROSSOVER":
#         crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 22.70)].iloc[0]
# except:
#     crossover_signal = None
# df_after_last_crossover

In [5]:
# raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()